<a href="https://colab.research.google.com/github/mrhamedani/LLM-Agents/blob/main/11_rouge_evaluations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/4-Evaluating%20LLMs/4_1_rouge_evaluations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>4.1-BLEU,  ROUGE and N-Grams. </h2>
    <h3>Evaluating summaries with ROUGE </h3>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)
_______
Models: t5-base-cnn / t5-base

Colab Environment: CPU

Keys:
* Summary Evaluation.
* N-Grams.
* Rouge.

Related article: [ROUGE Metrics: Evaluating Summaries in Large Language Models](https://pub.towardsai.net/rouge-metrics-evaluating-summaries-in-large-language-models-d200ee7ca0e6)
_______


# How to Evaluate Large Language Models for Summarization Using ROUGE.
The way we evaluate large language models is quite different from evaluating machine learning models, where metrics like Accuracy, F1 Score, or Recall were commonly used.

Metrics for generated language are distinct. Depending on the specific application, different metrics are chosen to assess the model's performance.

In this notebook, we will explore the usage of the ROUGE metric to measure the quality of summaries generated by a language model.

## What is ROUGE?
ROUGE isn't just a single metric; it's a set of metrics that measure the overlap and similarity between the generated summary and a reference summary that serves as a benchmark.

It returns fourth individual metrics. The metrics provided are:

* ROUGE-1: Measures the overlap of unigrams, or single words.
* ROUGE-2: Measures the overlap of bigrams, or pairs of words.
* ROUGE-L: Measures the longest common subsequence, rewarding longer shared sequences between the generated and reference summaries.
* ROUGE-LSUM: Calculated as the length of the LCS divided by the sum of the lengths of the generated summary and the reference summary.

## What are we going to do?
We are going to use two T5 models, one of them being the t5-Base model and the other a t5-base fine-tuned  specifically designed for creating summaries.

First, we will use a dataset and generate summaries using both models. By comparing the two generated summaries, we can observe whether the fine-tuning has been effective in producing different results. In other words, here we will only determine that the two models exhibit significant differences in summary generation, but we won't know which one might perform better.

To determine which model generates better summaries, we will utilize a well-known dataset called 'cnn_dailymail,' which is available in the 'datasets' library.

This dataset contains reference summaries that can be used for comparison. We will assess the summaries generated by the two models against these reference summaries.

The model that obtains a higher ROUGE score will be considered the one that produces better summaries.

## The models.
t5-Base Finnetunned: https://huggingface.co/flax-community/t5-base-cnn-dm

t5-Base: https://huggingface.co/t5-base


In [ ]:
!pip install -q evaluate==0.4.2
!pip install -q transformers==4.42.4
!pip install -q rouge_score==0.1.2
!pip install kaggle
#!pip install -q datasets==2.1.0

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Load the Data

In [ ]:
#Import generic libraries
import numpy as np
import pandas as pd
import torch


The dataset is available on Kaggle and comprises a collection of technological news articles compiled by MIT. The article text is located in the 'Article Body' column.

https://www.kaggle.com/datasets/deepanshudalal09/mit-ai-news-published-till-2023

## Importing Dataset from Kaggle

Yo only need acces to the articles.csv file from the Dataaset, you can download and load it directly, if you prefer to use the API Kaggle you can use the code Below. To use the Kaggle API you will need to hace your kaggle.json file with your keys in the directory /content/drive/MyDrive/kaggle

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [ ]:
!kaggle datasets download -d deepanshudalal09/mit-ai-news-published-till-2023

In [ ]:
import zipfile
file_path = '/content/mit-ai-news-published-till-2023.zip'
with zipfile.ZipFile(file_path, 'r') as zip_ref:
   zip_ref.extractall('/content/drive/MyDrive/kaggle')

## Loading Dataset

In [ ]:
news = pd.read_csv('/content/drive/MyDrive/kaggle/articles.csv')
DOCUMENT="Article Body"

In [ ]:
#Because it is just a course we select a small portion of News.
MAX_NEWS = 3
subset_news = news.head(MAX_NEWS)

In [ ]:
subset_news.head()

In [ ]:
articles = subset_news[DOCUMENT].tolist()

# Load the Models and create the summaries

Both models are available on Hugging Face, so we will work with the Transformers library.

In [ ]:
model_name_base = "t5-base"
model_name_finetuned = "flax-community/t5-base-cnn-dm"

In [ ]:
#This function returns the tokenizer and the Model.
def get_model(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    return tokenizer, model


In [ ]:
tokenizer_base, model_base = get_model(model_name_base)

In [ ]:
tokenizer_finetuned, model_finetuned = get_model(model_name_finetuned)

With both models downloaded and ready, we create a function that will perform the summaries.

The function takes fourth parameters:

* the list of texts to summarize.
* the tokenizer.
* the model.
* the maximum length for the generated summary

In [ ]:
import time

In [ ]:
def create_summaries(texts_list, tokenizer, model, max_l=125):

    # We are going to add a prefix to each article to be summarized
    # so that the model knows what it should do
    prefix = "Summarize this news: "
    summaries_list = [] #Will contain all summaries

    texts_list = [prefix + text for text in texts_list]

    for text in texts_list:

        summary=""

        #calculate the encodings
        input_encodings = tokenizer(text,
                                    max_length=1024,
                                    return_tensors='pt',
                                    padding=True,
                                    truncation=True)

        # Generate summaries
        start = time.time()
        output = model.generate(
            input_ids=input_encodings.input_ids,
            attention_mask=input_encodings.attention_mask,
            max_length=max_l,  # Set the maximum length of the generated summary
            num_beams=2,     # Set the number of beams for beam search
            early_stopping=True
        )

        #Decode to get the text
        summary = tokenizer.batch_decode(output, skip_special_tokens=True)
        end = time.time()
        #Add the summary to summaries list
        elapsed_time = end - start
        print(f"Time taken: {elapsed_time:.3f} seconds")
        summaries_list += summary
    return summaries_list


To create the summaries, we call the 'create_summaries' function, passing both the news articles and the corresponding tokenizer and model.

In [ ]:
# Creating the summaries for both models.
summaries_base = create_summaries(articles,
                                  tokenizer_base,
                                  model_base)


In [ ]:
summaries_finetuned = create_summaries(articles,
                                      tokenizer_finetuned,
                                      model_finetuned)

In [ ]:
summaries_base

In [ ]:
summaries_finetuned

At first glance, it's evident that the summaries are different.

However, it's challenging to determine which one is better.

It's even difficult to discern whether they are significantly distinct or if there are just subtle differences between them.

This is what we are going to verify now using ROUGE. When comparing the summaries of one model with those of the other, we don't get an idea of which one is better, but rather an idea of how much the summaries have changed with the fine-tuning applied to the model.

# ROUGE
Let's load the ROUEGE evaluator.

In [ ]:
#With the function load of the library evaluate
#we create a rouge_score object
rouge_score = evaluate.load("rouge")

Calculating ROUGE is as simple as calling the *compute* function of the *rouge_score* object we created earlier. This function takes the texts to compare as arguments and a third value *use_stemmer*, which indicates whether it should use *stemmer* or full words for the comparison.

A *stemmer* is the base of the word. Transform differents forms of a word in a same base.

Some samples of steammer are:
* Jumping -> Jump.
* Running -> Run.
* Cats -> Cat.

In [ ]:
def compute_rouge_score(generated, reference):

    #We need to add '\n' to each line before send it to ROUGE
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]

    return rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,

    )

In [ ]:
compute_rouge_score(summaries_base, summaries_finetuned)

We can see that there is a difference between the two models when performing summarization.

For example, in ROUGE-1, the similarity is 47%, while in ROUGE-2, it's a 32%. This indicates that the results are different, with some similarities but differents enough.

However, we still don't know which model is better since we have compared them to each other and not to a reference text. But at the very least, we know that the fine-tuning process applied to the second model has significantly altered its results.

# Comparing to a Dataset with real summaries.
We are going to load the Dataset cnn_dailymail. This is a well-known dataset available in the **Datasets** library, and it suits our purpose perfectly.

Apart from the news, it also contains pre-existing summaries.

We will compare the summaries generated by the two models we are using with those from the dataset to determine which model creates summaries that are closer to the reference ones.

In [ ]:
from datasets import load_dataset

cnn_dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0")

In [ ]:
#Get just a few news to test
sample_cnn = cnn_dataset["test"].select(range(MAX_NEWS))

sample_cnn

We retrieve the maximum length of the summaries to give the models the option to generate summaries of the same length, if they choose to do so.

In [ ]:
max_length = max(len(item['highlights']) for item in sample_cnn)
max_length = max_length + 10

In [ ]:
summaries_t5_base = create_summaries(sample_cnn["article"],
                                      tokenizer_base,
                                      model_base,
                                      max_l=max_length)

In [ ]:
summaries_t5_finetuned = create_summaries(sample_cnn["article"],
                                      tokenizer_finetuned,
                                      model_finetuned,
                                      max_l=max_length)

In [ ]:
#Get the real summaries from the cnn_dataset
real_summaries = sample_cnn['highlights']

Let's take a look at the generated summaries alongside the reference summaries provided by the dataset.

In [ ]:
summaries = pd.DataFrame.from_dict(
        {
            "base": summaries_t5_base,
            "finetuned": summaries_t5_finetuned,
            "reference": real_summaries,
        }
    )
summaries.head()

Now we can calculate the ROUGE scores for the two models.

In [ ]:
summaries_t5_base

In [ ]:
real_summaries

In [ ]:
compute_rouge_score(summaries_t5_base, real_summaries)

In [ ]:
compute_rouge_score(summaries_t5_finetuned, real_summaries)

With these results, I would say that the fine-tuned model performs slightly better than the T5-Base model. It consistently achieves higher ROUGE scores in all metrics except for LSUM, where the difference is minimal.

Additionally, the ROUGE metrics are quite interpretable.

LSUM indicates the percentage of the longest common subsequence, regardless of word order, in relation to the total length of the text.

This can be a good indicator of overall similarity between texts. However, both models have very similar LSUM scores, and the fine-tuned model has better scores in other ROUGE metrics.

Personally, I would lean towards the fine-tuned model, although the difference may not be very significant.


### Comparing entities with ROUGE

In [ ]:
entities=['Paris, Londres, Barcelona, Reus']
entities_ref=['Reus, Paris, Londres, Barcelona']

In [ ]:
compute_rouge_score(entities, entities_ref)

In [ ]:
entities_ref=['Paris, Londres, Barcelona, Reus']
compute_rouge_score(entities, entities_ref)